In [1]:
# Import packages needed for Predict Challenge
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [3]:
train = pd.read_csv('train.csv')
train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [4]:
from sklearn.utils import resample

In [5]:
pos_sentiment = train[train['sentiment']==1]
neg_sentiment = train[train['sentiment']==-1]
neutral = train[train['sentiment']==0]
news = train[train['sentiment']==2]

# Upsample minority
neg_sentiment_upsampled = resample(neg_sentiment, replace=True, n_samples=len(pos_sentiment), random_state=27)
neutral_upsampled = resample(neutral, replace=True, n_samples=len(pos_sentiment), random_state=27)
news_upsampled = resample(news, replace=True, n_samples=len(pos_sentiment), random_state=27)

In [6]:
# Combine Majority and Upsampled Minority Datasets
upsampled = pd.concat([pos_sentiment, neg_sentiment_upsampled, neutral_upsampled, news_upsampled])

In [7]:
y = upsampled['sentiment']
X = upsampled['message']

In [8]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2)
X_vectorized = vectorizer.fit_transform(X)

In [9]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=0.3,shuffle=True, random_state=27)

In [10]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_val)

In [11]:
f1_score(y_val, rfc_pred, average='macro')

0.9493390528000402

In [12]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [13]:
test = pd.read_csv('test.csv')
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


In [14]:
testx = test['message']
test_vect = vectorizer.transform(testx)

In [15]:
y_pred = rfc.predict(test_vect)

In [16]:
test['sentiment'] = y_pred

In [17]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


In [18]:
testsub = pd.DataFrame(y_pred, columns=['sentiment'])
testsub.head()

,sentiment
0,1
1,1
2,1
3,1
4,0


In [19]:
from google.colab import files
uploaded = files.upload()

Saving tweetid.csv to tweetid.csv


In [20]:
tweet = pd.read_csv('tweetid.csv')
tweet.head()

,tweetid
0,169760
1,35326
2,224985
3,476263
4,872928


In [21]:
output = pd.DataFrame({"tweetid":tweet['tweetid']})
testsubmission = output.join(testsub)
testsubmission.to_csv("testsubmission.csv", index=False)

In [22]:
testsubmission.head()

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,0
